In [ ]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
import pandas as pd
import rasterio
from rasterstats import zonal_stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from osgeo import gdal


In [ ]:
data1 = gpd.read_file('../../../../src/data_preprocessing/base_geojson/Telangana_grid1km.geojson')

In [ ]:
def meavaluesgrid(tif, data):
    mean_list = []
    for j in range(len(data)):                #iterating all rows of dataframe to get point info
        stats = zonal_stats(data.iloc[j].geometry, tif, stats="*", categorical=True)         #getting statistics from the raster point 
        i = stats[0]                                                                          #storing statsistical dictionary in a value
        mean = i['mean']
        mean_list.append(mean)
    return(mean_list)

In [ ]:
data = data1.reset_index()
#data = data.head(100)
soc_mandal = gpd.GeoDataFrame()
mean_list = meavaluesgrid('SoilOrganicCarbon/SOC_2020_0-5cm.tif', data)
soc_mandal['soc'] = mean_list
mean_list = meavaluesgrid('SoilOrganicCarbon/NDVI.tif',data)
soc_mandal['ndvi'] = mean_list
mean_list = meavaluesgrid('SoilOrganicCarbon/SoilBulkDensity_0-5cm.tif', data)
soc_mandal['bulkdensity'] = mean_list
mean_list = meavaluesgrid('SoilOrganicCarbon/ClayContent_0-5cm.tif', data)
soc_mandal['clay'] = mean_list
mean_list = meavaluesgrid('SoilOrganicCarbon/DEM.tif', data)
soc_mandal['dem'] = mean_list

soc_mandal['geometry'] = data.geometry

In [ ]:
soc_mandal = soc_mandal.dropna()

In [ ]:
import numpy as np

X = soc_mandal.values[:,1:5]
y =soc_mandal.soc

reg = LinearRegression().fit(X, y)
y_pred_trend = reg.predict(X)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

print('Regression Score: ',reg.score(X, y))
print('Mean Absolute Error: ',mean_absolute_error(y, y_pred_trend))
print('Root Mean Squared Error: ',mean_squared_error(y, y_pred_trend, squared=False))

In [ ]:
res = np.subtract(y,y_pred_trend)
#norm = (res - res.min())/ (res.max() - res.min())
data_norm = np.where(res >= 0, res/np.max(res), -res/np.min(res))
#data_norm1 = (norm - 0.5)/ 0.5
soc_mandal['y_pred'] = list(y_pred_trend)
soc_mandal['deviance'] = list(data_norm)

#base = soc_mandal.plot(color='white', edgecolor='black', figsize=(15,15))
#soc_mandal.plot(column='deviance', ax=base, cmap='RdYlGn', legend=True, legend_kwds={'label': "Deviance"}, figsize=(8,8))

In [ ]:
soc_mandal.to_file('soc_deviance1km.geojson')